## Affine gap global alignment with homopolymer indels

### Homopolymer indels
Some cutting-edge DNA sequencing technologies, such as Oxford Nanopore, perform poorly in regions of sequences that are *homopolymers*, which are runs of consecutive and identical bases.  For example, in the DNA sequence `GCTAGCCCCCTATC`, the substring `CCCCC` is a homopolymer of the base `C`.  Some sequencers have difficulty in accurately determining the length of such homopolymer regions.  If the example sequence just given were the true sequence, a sequencer could make a mistake and report a sequence of `GCTAGCCCTATC` (truncating the homopolymer region to three `C`s) or `GCTAGCCCCCCTATC` (expanding the homopolyer region to six `C`s), just to give a few examples of possible errors.

When aligning pairs of DNA sequences (say for the purpose of determining overlapping reads in genome assembly), a homopolymer error in a sequence results in an insertion or deletion in that sequence with respect to the other.  Given the knowledge that homopolymer errors are common in sequences from certain sequencing technologies, we may wish to use an alignment scoring scheme that penalizes gaps resulting from homopolymer errors *to a lesser degree* than other, non-homopolymer gaps.

### An algorithm for pairwise alignment with homopolymer indels
This implementation is a dynamic programming algorithm for global alignment with affine gap penalties that distinguishes between homopolymer and non-homopolymer insertions and uses different penalties for the two cases.  There are many ways in which this could be approached.  We will use one specific approach that considers a gap to be a *homopolymer gap* if the inserted sequence is a homopolymer and the base preceding the inserted sequence is the same as the base within the homopolymer.  Note that in pairwise alignment, a deletion in one sequence is indistinguishable from an insertion in the second sequence, and therefore we will consider all gaps to be insertions.  We will score a homopolymer gap of length $k$ using the  function:
$$w_h(k) = h + tk$$
and non-homopolymer gaps with the function
$$w(k) = g + sk,$$
and we will generally have that $h > g$ and $t > s$ such that homopolymer gaps penalized less than non-homopolymer gaps (note that all parameters are negative).

It turns out that we can find an optimal alignment with this scoring scheme using a modification of the standard affine gap global alignment dynamic programming algorithm.  Specifically, we will introduce two additional matrices, $H_x$ and $H_y$, which will keep track of alignments that end with a homopolymer insertion in $x$ and $y$, respectively.  To be precise, $H_x[i, j]$ will be defined as the optimal score of an alignment of the first $i$ characters of $x$ and the first $j$ characters of $y$ that ends in a homopolymer insertion in $x$.  The $H_x$ and $H_y$ matrices are similar to the $I_x$ and $I_y$ matrices in meaning, but are only for alignments ending in homopolymer gaps.  The dynamic programming recurrences thus become:

$M(i, j) = \max\left\{
\begin{array}{l}
M(i - 1, j - 1) + S(x_i, y_j) \\
I_x(i - 1, j - 1) + S(x_i, y_j) \\
I_y(i - 1, j - 1) + S(x_i, y_j) \\
H_x(i - 1, j - 1) + S(x_i, y_j) \\
H_y(i - 1, j - 1) + S(x_i, y_j) \\
\end{array}
\right.$

$I_x(i, j) = \max\left\{
\begin{array}{l}
M(i - 1, j) + g + s \\
I_x(i - 1, j) + s \\
\end{array}
\right.$

$I_y(i, j) = \max\left\{
\begin{array}{l}
M(i, j - 1) + g + s \\
I_y(i, j - 1) + s \\
\end{array}
\right.$

$H_x(i, j) = \delta(x[i] \neq x[i - 1]) + 
\max\left\{
\begin{array}{l}
M(i - 1, j) + h + t \\
H_x(i - 1, j) + t \\
\end{array}
\right.$

$H_y(i, j) = \delta(y[j] \neq y[j - 1]) +
\max\left\{
\begin{array}{l}
M(i, j - 1) + h + t \\
H_y(i, j - 1) + t \\
\end{array}
\right.$

In the recurrences for $H_x$ and $H_y$, the function $\delta$, is defined as 
$$\delta(condition) = \left\{
\begin{array}{ll}
-\infty, &\textrm{if}\ condition \\
0, & \textrm{otherwise}
\end{array}
\right.$$
which serves to restrict the scores in the $H$ matrices to those of alignments that end in homopolymer indels.

Note that the definition of a homopolymer gap provided above does *not* include insertions of a homopolymer at the beginning of either sequence, since there would be no base preceding such an insertion.  Therefore, for initialization, the entries of the first row and column of both the $H_x$ and $H_y$ matrices are set to $-\infty$.

This implementation takes as input two sequences, `x` and `y`, a substitution matrix, the gap scoring parameters ($g$, $h$, $s$, and $t$).  The substitution matrix will be represented as a dictionary with two-element tuples, `(a, b)`, as keys and scores as values. This implementation outputs a tuple of two elements, the first being the score of an optimal alignment, and the second being a single alignment that obtains that score.

In the case that there are multiple optimal alignments, during the traceback, if there are ties for which matrix to jump back to at each step, the order of preference for which matrix to jump to should be ($I_x$, $H_x$, $M$, $I_y$, $H_y$).  For example, in a $H_y$ cell, if the $M$ and $H_y$ cases in the recurrence both tie for the maximum, then during the traceback, you should prefer to jump back to the $M$ cell.

This implementation must use an efficient (polynomial-time) dynamic programming algorithm.

In [1]:
import align

def align_global_affine_hp_gaps(x, y, submatrix, g, h, s, t):
    """Computes an optimal global pairwise alignment 
    with an affine gap homopolymer-aware scoring function.
        
    Args:
        x: a string representing the first sequence
        y: a string representing the second sequence
        submatrix: a substitution matrix
        g: the gap existence score for non-homopolymer gaps
        h: the gap existence score for homopolymer gaps
        s: the space score for non-homopolymer gaps
        t: the space score for homopolymer gaps
    Returns:
        A tuple, (score, alignment), where score is a numeric value giving the score of the
        alignment and alignment is a list of two strings
    """
    return align.align_global_affine_hp_gaps(x, y, submatrix, g, h, s, t)